In [ ]:
#| hide
from poniard.core import *

# poniard

> Streamline scikit-learn model comparison.

This file will become your README and also the index of your documentation.

## Install

```sh
pip install poniard
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2